In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from plot_style.style_prab import load_preset,figsize,cmap_nicify
from plot_style.style_prab import colors as colors_preset
load_preset(scale=1,font_path='../plot_style/font')
import pickle
mymap = cmap_nicify(cmap='YlGnBu_r',idx_white=1,size_white=50)
from scipy.interpolate import interp1d

from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)


In [ ]:
path = '../data/dataframe_combined_xspec_interpolated_gaia_energy_2022.h5'
df_xspec = pd.read_hdf(path)[:-1]

path = '../data/dataframe_combined_espec_interpolated_gaia_energy_2022.h5'
df_espec = pd.read_hdf(path)[:-1]

wl = np.load('../data/wavelength_axis.npy')

In [ ]:
# check if diagnostics are synced
f = df_espec['bpm2_q'] > 20
f = df_xspec['bpm2_q'] > 20
nonan = ~df_espec['espec_high_energy_median'].isna() & ~df_xspec['xspec_1st_order_wavelangth'].isna()
f = f & nonan

x = df_espec['espec_high_energy_median'][f]
y = df_espec['xspec_1st_order_wavelangth'][f]

plt.hexbin(x,y,cmap=mymap,bins=500, extent = (235,290,9,14)) #extent = (1,15,1,3),bins=50)
plt.xlabel('Median absolute deviation')
plt.ylabel('1st order width')

In [ ]:
# remove all nan-shots from xspec-data
nonan = ~pd.isna(df_xspec['xspec_spectrum'])
df_xspecx = df_xspec['xspec_spectrum'][nonan].to_frame()

# join espec and xspec on shotid
joint = df_espec.join(df_xspecx,on='shotid')
joint = joint.sort_values(by='espec_high_energy_median',ascending=True)
nonan1 = ~pd.isna(joint['xspec_1st_order_wavelangth'])
#nonan2 = ~pd.isna(joint['3'])


ff = joint['amp2_energy'] > 5.5
fff = joint['saga3_energy'] > 1
ffff = joint['oap_ff_peak_x']> -400
f = nonan1 & ff & fff & ffff
joint = joint[f]

nonan = ~pd.isna(joint['xspec_1st_order_wavelangth'])
joint = joint[nonan]
nonan = joint['bpm1_q']>20
joint = joint[nonan]
#nonan = joint['espec_high_energy_median']<260
#joint = joint[nonan]

In [ ]:
# create bins
n = 100
m = np.nanmean(joint['espec_high_energy_median'].values)
s = np.nanstd(joint['espec_high_energy_median'].values)
counts,bins,_ = plt.hist(joint['espec_high_energy_median'].values,bins=n,range=[m-2.5*s,m+2.5*s])

In [ ]:
wfp = np.zeros((n+1,1024//2-300))
y = []
mad = np.zeros(n+1,)

for i,row in enumerate(joint.iterrows()):
    E,wl_spec,mad_ = row[1]['espec_high_energy_median'],row[1]['xspec_spectrum'],row[1]['espec_high_energy_mad']
    for j in range(1,bins.shape[0]):
        if E>bins[j-1] and E<bins[j]:
            m = np.nansum(wl_spec[1024//2+12-50:1024//2+12+50])*(wl[1]-wl[0])
            #m = np.nansum(wl_spec)*(wl[1]-wl[0])
            #if m < 1000:
            #    break
            wfp[j,:] += wl_spec[1024//2:-300]/m
            mad[j] += mad_
            break
wfp = wfp[1:,:]
mad= mad[1:]
wfp = wfp/counts[:,np.newaxis]
mad = mad/counts

In [ ]:
wl = np.load('../data/wavelength_axis.npy')
y = bins[1:]-np.diff(bins)[0]/2
plt.pcolormesh(wl[1024//2+12:-300]-wl[1024//2+12],y,wfp[:,12:],cmap=mymap,vmin=0,vmax=0.5)

wl_u = 5e6 # nm
K = 0.29 #0.29 # unitless
mec2 = 0.511 #MeV 

# undulator eq
def xray_wl(E):
    return (wl_u*mec2**2/(2*(E)**2))*(1+K**2/2) 

E = np.linspace(240,290)
xwl = xray_wl(E)
plt.plot(xwl,E,'k',label='Undulator eq.')


In [ ]:
d = {'mad':mad,
     'E' : y,
     'wl' : wl[1024//2+12:-300]-wl[1024//2+12],
     'wfp' : wfp[:,12:],}
with open('../data/waterfall_data_xray_normalized.pkl','wb') as fp:
    pickle.dump(d,fp)

In [ ]:
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize


fig,ax = plt.subplots(1,1)

mad_target = np.arange(242,260,1)
E = bins[1:]-np.diff(bins)[0]/2
for m in mad_target:
    i = np.where(E>m)[0][0]
    c = (m-230)/(260-230) 
    color = plt.cm.gnuplot(c)
    ii = np.argmax(wfp[i,50:])

    x = wl[1024//2+50:-300]-wl[1024//2+12]
    x_0 = x[ii]
    x = x-x_0
    ax.plot(x,wfp[i,50:]/wfp[i,50:].max(),color=color)


cmappable = ScalarMappable(norm=Normalize(230,260), cmap='gnuplot')
fig.colorbar(cmappable, ax=ax)
plt.xlabel('Wavelength [nm]')
plt.title('Xray-spectrum (MAD values diffrentiated by color) \n moving average, Normalized to max value')
#plt.savefig('xray_spectrum_mad_averaged_normalized_low_energy_shifted.png',dpi=300)